### Introduction

Link/edge prediction demo for cora dataset (homegeneous network) where all nodes are papers and edges between nodes are citation links, e.g., paper A cites paper B. 

Each paper has a **subject** attribute with one of 7 values denoting the subject area of the paper.

This demo notebook **demonstrates how to predict citation links/edges between papers** using the random walk-based representation learning method Node2Vec.

**References:** 

**[1]** Node2Vec: Scalable Feature Learning for Networks. A. Grover, J. Leskovec. ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD), 2016. 

In [ ]:
import matplotlib.pyplot as plt
from math import isclose
from sklearn.decomposition import PCA

import os
import networkx as nx
import numpy as np
import pandas as pd

from stellargraph.data import EdgeSplitter
from stellargraph.data import BiasedRandomWalk
from stellargraph import StellarGraph

from sklearn.metrics import roc_auc_score
from collections import Counter
import multiprocessing

%matplotlib inline

In [ ]:
# Default parameters for Node2Vec
parameters = {
    "p": 1.,  # Parameter p
    "q": 1.,  # Parameter q
    "dimensions": 128,  # dimensionality of node2vec embeddings
    "num_walks": 10,  # Number of walks from each node
    "walk_length": 80,  # Walk length
    "window_size": 10,  # Context size for word2vec
    "iter": 1,  # number of SGD iterations (epochs)
    "workers": multiprocessing.cpu_count(),  # number of workers for word2vec
    "weighted": False,  # is graph weighted?
    "directed": False,  # are edges directed?
}

### Load the dataset

The dataset is the citation network Cora.

It can be downloaded by clicking [here](https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz)

The following is the description of the dataset from the publisher,

> The Cora dataset consists of 2708 scientific publications classified into one of seven classes. The citation network consists of 5429 links. Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary. The dictionary consists of 1433 unique words. The README file in the dataset provides more details. 

For this demo, we ignore the word vector associated with each paper. We are only interested in the network structure.

Download and unzip the cora.tgz file to a location on your computer. For the purposes of this demo, we are only interested in the **cora.cites** file that is a representation of the graph as an edge list.

We assume that the dataset is stored in the directory

`../data/cora/cora.cites`

In [ ]:
data_dir = "../data/cora"

In [ ]:
cora_location = os.path.expanduser(os.path.join(data_dir, "cora.cites"))
g_nx = nx.read_edgelist(path=cora_location)

# load the node attribute data
cora_data_location = os.path.expanduser(os.path.join(data_dir, "cora.content"))
node_attr = pd.read_csv(cora_data_location, sep='\t', header=None)
values = { str(row.tolist()[0]): row.tolist()[-1] for _, row in node_attr.iterrows()}
nx.set_node_attributes(g_nx, values, 'subject')

# Select the largest connected component. For clarity we ignore isolated
# nodes and subgraphs; having these in the data does not prevent the
# algorithm from running and producing valid results.
g_nx_ccs = (g_nx.subgraph(c).copy() for c in nx.connected_components(g_nx))
g_nx = max(g_nx_ccs, key=len)
print("Largest subgraph statistics: {} nodes, {} edges".format(
    g_nx.number_of_nodes(), g_nx.number_of_edges()))

### Construct train and test splits of the input data

We are going to tackle link prediction as a supervised learning problem whereas we are going to train a binary classifier to predict a link, or not, between to nodes in the graph. 

For supervised classification, we need a training set of positive and negative examples of links and a corresponding **train** graph with the positive links removed; this graph is used for representation learning.

For evaluation, we will use a hold out dataset, that is a set of positive and negative examples of links and a corresponding **test** graph.

We are going to split our input graph into a **train** and **test** graphs using the EdgeSplitter class in stellar.data.edge_splitter. We are going to use the **train** graph for training the predictive model (a binary classifier that given two nodes it will predict whether a link between these two nodes should exist or not) and the **test** graph for evaluating the predictive model's performance on hold out data. 

Each graph will have the same number of nodes as the input graph but the number of edges will differ as some of the edges will be removed during each split and used as the positive samples for training the link prediction classifier.

In [ ]:
# Test graph and edge test data
edge_splitter_test = EdgeSplitter(g_nx)
g_test, edge_data_ids_test, edge_data_labels_test = edge_splitter_test.train_test_split( 
    p=0.15, method='global'
)

# Train graph and edge train data
edge_splitter_val = EdgeSplitter(g_test, g_nx)
g_val, edge_data_ids_val, edge_data_labels_val = edge_splitter_val.train_test_split(
    p=0.15, method='global'
)

# Train graph and edge train data
edge_splitter_train = EdgeSplitter(g_val, g_nx)
g_train, edge_data_ids_train, edge_data_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method='global'
)

### Convert graphs

The node2vec reference implementation requires that the given graph is undirected and homogeneous. We cast the **train** and **test** graphs accordingly.

In [ ]:
# Train and Test graphs should be of type nx.Graph
g_test = nx.Graph(g_test)
g_val = nx.Graph(g_val)
g_train = nx.Graph(g_train)

### Train the link prediction model and evaluate on test data

We are going to train a link prediction classifier in three steps. 
1. We use Node2Vec, [1], to calculate node embeddings. These embeddings are learned in such a way to ensure that nodes that are close in the graph remain close in the embedding space.
2. We calculate link/edge embeddings for the positive and negative edge samples by applying a binary operator on the embeddings of the source and target nodes of each sampled edge. We consider 4 different operators, hadamard, l1, l2, and average; the paper in [1] provides a detailed description of these operators. All operators produce link embeddings that have equal dimensionality to the input node embeddings (128 dimensions for our example.) 
3. Given the embeddings of the positive and negative examples, we train a logistic regression classifier to predict a binary value indicating whether an edge between two nodes should exist or not.
4. We evaluate the performance of the link classifier for each of the 4 operators on the training data and select the best performing one. The selected operator is used to predict the hold out, test, data with node embeddings calculated on the **test** graph.


In [ ]:
random_walk_length = 100
num_walks_per_node = 10
p = 1.
q = 1.

In [ ]:
rw = BiasedRandomWalk(StellarGraph(g_train))

walks = rw.run(nodes=list(g_train.nodes()), # root nodes
               length=random_walk_length,  # maximum length of a random walk
               n=num_walks_per_node,        # number of random walks per root node 
               p=p,       # Defines (unormalised) probability, 1/p, of returning to source node
               q=q        # Defines (unormalised) probability, 1/q, for moving away from source node
              )
print("Number of random walks: {}".format(len(walks)))

In [ ]:
embedding_dim = 128 
context_window_size = 5

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(walks, size=embedding_dim, window=context_window_size, min_count=0, sg=1, workers=2, iter=1)

In [ ]:
edge_data_ids_train.shape

In [ ]:
def operator_hadamard(u, v):
    return u*v

def operator_avg(u, v):
    return (u+v)/2.0

def operator_l2(u, v):
    return (u-v)**2

def operator_l1(u, v):
    return np.abs(u-v)

def transform(model, edge_data, binary_operator):
    """
    It calculates edge features for the given binary operator applied to the node features in data_edge
    :param edge_data: (2-tuple) It is a list of pairs of nodes that make an edge in the graph
    :param binary_operator: The binary operator to apply to the node features to calculate an edge feature
    :return: Features in X (Nxd array where N is the number of edges and d is the dimensionality of the edge
        features that is the same as the dimensionality of the node features) and edge labels in y (0 for no edge
        and 1 for edge).    
    """
    X = []  # data matrix, each row is a d-dimensional feature of an edge
        
    for ids in edge_data:        
        X.append(binary_operator(model[ids[0]], model[ids[1]]))
  
    return np.array(X)

In [ ]:
edge_train_features = transform(model=model, 
                                edge_data=edge_data_ids_train, 
                                binary_operator=operator_l2)
edge_val_features = transform(model=model, 
                              edge_data=edge_data_ids_val, 
                              binary_operator=operator_l2)

In [ ]:
edge_train_features.shape, edge_val_features.shape

### Visualise representations of link embeddings

Learned link embeddings have 128 dimensions but for visualisation we project them down to 2 dimensions using the PCA algorithm ([link](https://en.wikipedia.org/wiki/Principal_component_analysis)). 

Green points represent positive edges and red points represent negative (no edge should exist between the corresponding vertices) edges.

In [ ]:
# Calculate edge features for validation data
edge_data = (edge_data_ids_val, edge_data_labels_val)
X, y = edge_val_features, edge_data_labels_val

# Learn a projection from 128 dimensions to 2
pca = PCA(n_components=2)
X_transformed = pca.fit_transform(X)

# plot the 2-dimensional points
red = y == 0
green = y == 1
plt.figure(figsize=(16,12))
plt.scatter(X_transformed[red, 0], X_transformed[red, 1], c="r", alpha=0.8)
plt.scatter(X_transformed[green, 0], X_transformed[green, 1], c="g", alpha=0.4)

## Classification

We can use the edge features to train a classifier. We can then use this classifier to predict the test set and measure its generalisation performance.

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
clf = LogisticRegressionCV( Cs=10, cv=5, scoring='roc_auc', max_iter=1000, verbose=False)

In [ ]:
clf.fit(edge_train_features, edge_data_labels_train)

In [ ]:
train_predictions = clf.predict_proba(edge_train_features)
if clf.classes_[0]==1:
    train_predictions = train_predictions[:, 0]
else:
    train_predictions = train_predictions[: ,1]

In [ ]:
roc_auc_score(edge_data_labels_train, train_predictions)

In [ ]:
val_predictions = clf.predict_proba(edge_val_features)
if clf.classes_[0]==1:
    val_predictions = val_predictions[:, 0]
else:
    val_predictions = val_predictions[: ,1]

In [ ]:
roc_auc_score(edge_data_labels_val, val_predictions)

### Representation learning on the validation graph

We are going to repeat the representation learning step on the validation graph because it differs from the train graph.

**Note:** We are not going to retrain the classifier. 

In [ ]:
rw = BiasedRandomWalk(StellarGraph(g_val))

walks = rw.run(nodes=list(g_val.nodes()), # root nodes
               length=random_walk_length,  # maximum length of a random walk
               n=num_walks_per_node,        # number of random walks per root node 
               p=p,       # Defines (unormalised) probability, 1/p, of returning to source node
               q=q        # Defines (unormalised) probability, 1/q, for moving away from source node
              )
print("Number of random walks: {}".format(len(walks)))

model = Word2Vec(walks, size=embedding_dim, window=context_window_size, min_count=0, sg=1, workers=2, iter=1)

### Predict the test data

In [ ]:
edge_test_features = transform(model=model, 
                              edge_data=edge_data_ids_test, 
                              binary_operator=operator_l2)

In [ ]:
test_predictions = clf.predict_proba(edge_test_features)
if clf.classes_[0]==1:
    test_predictions = test_predictions[:, 0]
else:
    test_predictions = test_predictions[: ,1]

In [ ]:
roc_auc_score(edge_data_labels_test, test_predictions)